In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
news = pd.read_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/NEWS_+_PRESS_ESG_definitivo.csv')
esg_scores = pd.read_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/ESG_SCORES_COMPLETOS_2.csv')
oficial_esg_scores = pd.read_csv('/content/drive/MyDrive/MIAX/TFM/AAA RED NEURONAL/ESG_SCORES.csv')

In [ ]:
esg_scores

,date,ticker,category,predicted_score,predicted_score_final,from_official
0,2020-03-23,AAPL,Environmental,NaN,NaN,True
1,2020-03-24,AAPL,Environmental,NaN,NaN,True
2,2020-03-25,AAPL,Environmental,NaN,NaN,True
3,2020-03-26,AAPL,Environmental,NaN,NaN,True
4,2020-03-27,AAPL,Environmental,NaN,NaN,True
...,...,...,...,...,...,...
164425,2025-03-19,XOM,Governance,70.64299,70.64299,False
164426,2025-03-20,XOM,Governance,NaN,70.64299,False
164427,2025-03-21,XOM,Governance,NaN,70.64299,False
164428,2025-03-22,XOM,Governance,70.34622,70.34622,False


In [ ]:
oficial_esg_scores

,date,acceptedDate,symbol,cik,companyName,formType,environmentalScore,socialScore,governanceScore,ESGScore,url
0,2024-12-28,2025-01-30,AAPL,320193,Apple Inc.,8-K,52.52,45.18,60.74,52.81,https://www.sec.gov/Archives/edgar/data/320193...
1,2024-09-28,2024-11-01,AAPL,320193,Apple Inc.,10-K,68.47,47.02,60.80,58.76,https://www.sec.gov/Archives/edgar/data/320193...
2,2024-06-29,2024-08-01,AAPL,320193,Apple Inc.,8-K,52.37,44.70,60.48,52.52,https://www.sec.gov/Archives/edgar/data/320193...
3,2024-03-30,2024-05-02,AAPL,320193,Apple Inc.,10-Q,72.07,49.55,61.63,61.08,https://www.sec.gov/Archives/edgar/data/320193...
4,2023-12-30,2024-02-01,AAPL,320193,Apple Inc.,10-Q,72.07,49.38,61.51,60.99,https://www.sec.gov/Archives/edgar/data/320193...
...,...,...,...,...,...,...,...,...,...,...,...
523,2021-03-31,2021-04-29,AMZN,1018724,"Amazon.com, Inc.",10-Q,50.14,58.90,53.16,54.07,https://www.sec.gov/Archives/edgar/data/101872...
524,2020-12-31,2021-02-02,AMZN,1018724,"Amazon.com, Inc.",10-K,50.33,58.75,54.40,54.50,https://www.sec.gov/Archives/edgar/data/101872...
525,2020-09-30,2020-10-29,AMZN,1018724,"Amazon.com, Inc.",10-Q,49.38,58.47,53.18,53.68,https://www.sec.gov/Archives/edgar/data/101872...
526,2020-06-30,2020-07-30,AMZN,1018724,"Amazon.com, Inc.",10-Q,49.10,58.73,53.61,53.82,https://www.sec.gov/Archives/edgar/data/101872...


In [ ]:
import numpy as np

In [ ]:
esg_scores["date"] = pd.to_datetime(esg_scores["date"])
oficial_esg_scores["date"] = pd.to_datetime(oficial_esg_scores["date"])

oficial_long = oficial_esg_scores.melt(
    id_vars=["date", "symbol"],
    value_vars=["environmentalScore", "socialScore", "governanceScore"],
    var_name="category",
    value_name="score"
)

oficial_long["category"] = oficial_long["category"].str.replace("Score", "")
oficial_long["category"] = oficial_long["category"].str.capitalize()
oficial_long.rename(columns={"symbol": "ticker"}, inplace=True)

oficial_long.set_index(["ticker", "category"], inplace=True)

esg_scores = esg_scores.sort_values(["ticker", "category", "date"]).reset_index(drop=True)

esg_scores["predicted_score_final"] = (
    esg_scores.groupby(["ticker", "category"])["predicted_score_final"]
    .ffill()
)

def buscar_score_mas_cercano(row):
    if pd.notna(row["predicted_score_final"]):
        return row["predicted_score_final"]

    key = (row["ticker"], row["category"])
    if key not in oficial_long.index:
        return np.nan

    sub_df = oficial_long.loc[key].reset_index()
    if sub_df.empty:
        return np.nan

    sub_df["diff"] = (sub_df["date"] - row["date"]).abs()
    closest = sub_df.loc[sub_df["diff"].idxmin()]
    return closest["score"]

esg_scores["predicted_score_final"] = esg_scores.apply(buscar_score_mas_cercano, axis=1)

Se han truncado las últimas 5000 líneas del flujo de salida.
<ipython-input-7-3dfe6a2f5400>:29: PerformanceWarning: indexing past lexsort depth may impact performance.
  if key not in oficial_long.index:
<ipython-input-7-3dfe6a2f5400>:32: PerformanceWarning: indexing past lexsort depth may impact performance.
  sub_df = oficial_long.loc[key].reset_index()
<ipython-input-7-3dfe6a2f5400>:29: PerformanceWarning: indexing past lexsort depth may impact performance.
  if key not in oficial_long.index:
<ipython-input-7-3dfe6a2f5400>:32: PerformanceWarning: indexing past lexsort depth may impact performance.
  sub_df = oficial_long.loc[key].reset_index()
<ipython-input-7-3dfe6a2f5400>:29: PerformanceWarning: indexing past lexsort depth may impact performance.
  if key not in oficial_long.index:
<ipython-input-7-3dfe6a2f5400>:32: PerformanceWarning: indexing past lexsort depth may impact performance.
  sub_df = oficial_long.loc[key].reset_index()
<ipython-input-7-3dfe6a2f5400>:29: PerformanceWa

In [ ]:
esg_scores[esg_scores['ticker']=='MCD']

,date,ticker,category,predicted_score,predicted_score_final,from_official
82215,2020-03-23,MCD,Environmental,NaN,57.500000,True
82216,2020-03-24,MCD,Environmental,NaN,57.500000,True
82217,2020-03-25,MCD,Environmental,NaN,57.500000,True
82218,2020-03-26,MCD,Environmental,NaN,57.500000,True
82219,2020-03-27,MCD,Environmental,NaN,57.500000,True
...,...,...,...,...,...,...
87691,2025-03-19,MCD,Social,NaN,62.319202,False
87692,2025-03-20,MCD,Social,NaN,62.319202,False
87693,2025-03-21,MCD,Social,NaN,62.319202,False
87694,2025-03-22,MCD,Social,NaN,62.319202,False


In [ ]:
prices = pd.read_csv('/content/drive/MyDrive/MIAX/TFM/AAA RED NEURONAL/PRICES.csv')

In [ ]:
prices

,ticker,date,open,high,low,close,volume,change,changePercent,vwap
0,AAPL,2025-03-25,220.81,223.949,220.08,223.07,20365126,2.26,1.023500,222.3700
1,AAPL,2025-03-24,221.00,221.480,218.58,220.73,44172627,-0.27,-0.122170,220.4475
2,AAPL,2025-03-21,211.56,218.840,211.28,218.27,94127768,6.71,3.170000,214.9875
3,AAPL,2025-03-20,213.99,217.490,212.22,214.10,48862947,0.11,0.051404,214.4500
4,AAPL,2025-03-19,214.22,218.760,213.75,215.24,54385400,1.02,0.476150,215.4925
...,...,...,...,...,...,...,...,...,...,...
38575,JNJ,2020-04-01,127.70,130.230,125.50,128.81,12606081,1.11,0.869220,128.0600
38576,JNJ,2020-03-31,134.99,135.710,130.05,131.13,15746124,-3.86,-2.860000,132.9700
38577,JNJ,2020-03-30,128.46,133.990,127.50,133.01,21754033,4.55,3.540000,130.7400
38578,JNJ,2020-03-27,121.65,127.620,120.58,123.16,12133222,1.51,1.240000,123.2525


In [ ]:
valid_tickers = esg_scores["ticker"].unique()
news = news[news["ticker"].isin(valid_tickers)].copy()
prices = prices[prices["ticker"].isin(valid_tickers)].copy()

In [ ]:
prices["date"] = pd.to_datetime(prices["date"])
fecha_minima = prices["date"].min()

In [ ]:
news["date"] = pd.to_datetime(news["date"])
esg_scores["date"] = pd.to_datetime(esg_scores["date"])

fecha_inicio = pd.Timestamp("2020-03-26")
news = news[news["date"] >= fecha_inicio].reset_index(drop=True)
esg_scores = esg_scores[esg_scores["date"] >= fecha_inicio].reset_index(drop=True)

In [ ]:
news.to_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/news_clasificacion_final.csv', index=False)
esg_scores.to_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/esg_scores_final.csv', index=False)
prices.to_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/prices_final.csv', index=False)